# Твиты

1) В качестве брокера сообщений использовать Kafka.

2) Для работы с потоком твитов - пакет tweepy под Python.

## Подготовка

## Задача 5. Подсчет количества ретвитов на твиты пользователей

Напишите программу, которая подсчитывает количество ретвитов на твиты каждого пользователя в течение 1 мин. с накоплением.

Выведите результат в отсортированном по убыванию виде: id твита, количество ретвитов. Каждую 1 мин. сохранять результат в файл. Накапливать не менее 15 мин.

После этого для 5 наиболее популярных твитов вывести: screen_name пользователя, id твита и его текст.

Исходные данные:
- id пользователей: "285532415", "147964447", "34200559", "338960856", "200036850", "72525490", "20510157", "99918629"

Чуть-чуть подправим наш "производитель твитов": вынесем AUTH-данные в отдельный файл и добавим вывод ID твита.

**local_settings.py**

```python
CONSUMER_TOKEN = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
```

**tweets_producer.py**

```python
import logging
import sys

import rapidjson
import tweepy

from tweepy.streaming import json
from kafka import KafkaProducer

from local_settings import *

logger = logging.getLogger('tweets_producer')
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('[%(levelname)s] %(asctime)s  %(name)s: %(message)s'))
logger.addHandler(handler)


KAFKA_SERVERS = ('localhost:9092',)
TWEETS_KAFKA_TOPIC = 'Tweets'

USERS_IDS = ('285532415', '147964447', '34200559', '338960856', '200036850', '72525490', '20510157', '99918629')


class BaseListener(tweepy.StreamListener):

    def __init__(self):
        self.kafka_producer = KafkaProducer(bootstrap_servers=KAFKA_SERVERS)

    def on_status(self, status):
        logger.warning('Status: %s', status.text)

    def on_error(self, status_code):
        logger.warning(f'Error: %s', status_code)


class TweetsStreamListener(BaseListener):

    def on_data(self, raw_data):
        data = rapidjson.loads(raw_data)

        tid = data.get('id', None)
        if not tid:
            logger.warning('Tweet ID was not found in data: %s', raw_data)
            return True

        user = data.get('user', None)
        if not user:
            logger.warning('User was not found in data: %s', raw_data)
            return True

        screen_name, uid = user['screen_name'], user['id']

        logger.info('Received tweet #%s from @%s[%s]', tid, screen_name, uid)

        self.kafka_producer.send(TWEETS_KAFKA_TOPIC,
            rapidjson.dumps({
                'tid': tid,
                'screen_name': screen_name,
            }).encode('utf-8')
        )
        return True


def main():
    auth = tweepy.OAuthHandler(CONSUMER_TOKEN, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)

    tweets_stream = tweepy.Stream(auth=api.auth, listener=TweetsStreamListener())
    logger.info('Start tweets receiving... Use kafka topic `%s`', TWEETS_KAFKA_TOPIC)
    tweets_stream.filter(follow=USERS_IDS)


if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        logger.warning('Was stopped by user request')

```

В "счетчике ретвитов" теперь используется *tweepy* для получения актуальной информации о твите.

**retweets_counter.py**

```python
import sys
import time

from datetime import datetime

import rapidjson
import tweepy

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

from local_settings import *

ZOOKEEPER_SERVER = 'localhost:2181'
KAFKA_TOPIC = 'Tweets'

APP_NAME = 'UsersRetweetsCounter'
MINUTE = 60
BATCH_DURATION_SEC = MINUTE

# Init twitter API
auth = tweepy.OAuthHandler(CONSUMER_TOKEN, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

# Init streaming
spark_context = SparkContext(appName=APP_NAME)
spark_context.setLogLevel('ERROR')

streaming_context = StreamingContext(spark_context, BATCH_DURATION_SEC)
streaming_context.checkpoint(f'{APP_NAME}__checkpoint')

# Process tweets
kafka_stream = KafkaUtils.createStream(
	streaming_context,
	ZOOKEEPER_SERVER,
	f'{APP_NAME}__consumers_group',
	{KAFKA_TOPIC: 1},
)

# [
#	(None, '{"tid":1074310692235292672,"screen_name":"Odev110"}'),
#	(None, '{"tid":1074310709134139392,"screen_name":"kirillica1957"}')
#	(None, '{"tid":1074310692235292673,"screen_name":"Odev110"}')
# ] => [
#	'1074310692235292672',
# 	'1074310709134139392',
# 	'1074310692235292673'
# ]
tweets_ids = kafka_stream.map(lambda data: rapidjson.loads(data[1])['tid'])

# [
#	'1074310692235292672',
# 	'1074310709134139392',
# 	'1074310692235292673'
# ] => [
# 	('1074310692235292672', '1074310692235292672'),
#	('1074310709134139392', '1074310709134139392'),
#	('1074310692235292673', '1074310692235292673'),
# ]
tid__retweets_count = tweets_ids.map(lambda tid: (tid, tid))  # Further -> (tid, (tid, retweets count))


def update_retweets_count(new_tids, tid__retweets_count__in_state):
	tid = new_tids[0] if new_tids else tid__retweets_count__in_state[0]
	
	retweet_count = tid__retweets_count__in_state[1] if tid__retweets_count__in_state else 0
	try:
		retweet_count = api.get_status(tid).retweet_count
	except tweepy.error.TweepError:
		# print(f'Tweet #{tid} was deleted')
		pass

	return (tid, retweet_count)

	# TODO(a.telyshev): More interesting variant, but "tweepy.error.RateLimitError":
	#                   https://developer.twitter.com/en/docs/basics/rate-limits.html
	# retweets_count = (
	#	tid__retweets_count__in_state[1]
	#	if tid__retweets_count__in_state else 0
	# )
	# now = datetime.now()
	# retweets = [
	# 	tweet
	# 	for tweet in api.retweets(tid)
	# 	if (now - tweet.created_at).total_seconds() < BATCH_DURATION_SEC  # Skip already processed retweets
	# ]
	# for tweet in retweets:
	# 	print(f'New retweet #{tweet.id}({tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")}) for tweet #{tid}')
	# 
	# return (tid, retweets_count + len(retweets))


tid__retweets_total_count = tid__retweets_count.updateStateByKey(update_retweets_count)

top_tweets_by_retweets = (
	tid__retweets_total_count
		.transform(
			lambda rdd: rdd.sortBy(lambda tid___tid__retweets_count: -tid___tid__retweets_count[1][1])
		)
		.map(lambda tid___tid__retweets_count: tid___tid__retweets_count[1])
	)

# Every minute print 10 of top retweeted tweets and save result in file
top_tweets_by_retweets.pprint(10)
top_tweets_by_retweets.transform(lambda rdd: rdd.coalesce(1)).saveAsTextFiles('file:///tmp/retweets/counts')

# Every 15 minutes print info about 5 of top retweeted tweets
windowed_data = top_tweets_by_retweets.reduceByKeyAndWindow(
	max,
	None,
	windowDuration=MINUTE * 15,
	slideDuration=MINUTE * 15
)


def get_and_print_top_tweets_info(rdd, limit=5):
	for tid__retweets_count in (
		rdd
			.sortBy(lambda tid__retweets_count: -tid__retweets_count[1])
			.take(limit)
	):
		tid = tid__retweets_count[0]
		try:
			tweet = api.get_status(tid)
		except tweepy.error.TweepError:
			print(f'Tweet #{tid} was deleted')
			return

		print(f'\n@{tweet.user.screen_name} tweeted in #{tid}:\n"""\n{tweet.text}\n"""\n')


windowed_data.transform(lambda rdd: rdd.coalesce(1)).foreachRDD(lambda _, rdd: get_and_print_top_tweets_info(rdd))

# Start and deinit later
streaming_context.start()
streaming_context.awaitTermination()

```

Мы видим, что за 15 минут у твита **#1074429629405061132** появился один новый ретвит (636 -> 637).
Также можно проследить динамику роста ретвитов у твита **#1074430927189786624** (2 -> 3 -> 4 -> 5 -> 6) в течении 5 минут. 

```text
-------------------------------------------
Time: 2018-12-16 14:28:00
-------------------------------------------
...
(1074430927189786624, 2)
...

-------------------------------------------
Time: 2018-12-16 14:29:00
-------------------------------------------
...
(1074430927189786624, 3)
...

-------------------------------------------
Time: 2018-12-16 14:30:00
-------------------------------------------
...
(1074430927189786624, 4)
...

-------------------------------------------
Time: 2018-12-16 14:31:00
-------------------------------------------
...
(1074430927189786624, 5)
...

-------------------------------------------
Time: 2018-12-16 14:32:00
-------------------------------------------
...
(1074430927189786624, 6)
...
```

Проверим информацию о первом и последнем твитах нашего ТОП-5.

```text
-------------------------------------------
Time: 2018-12-16 14:38:00
-------------------------------------------
(1074429629405061132, 637)
...
(1074430774017966082, 65)
```

**#1074429629405061132**
```text
@QTOgNFPGUy6iW5r tweeted in #1074429629405061132:
"""
RT @tass_agency: В Сочи созрел рекордный урожай киви. Оказывается, тропический фрукт в Краснодарском крае выращивают уже более 30 лет.

Вид…
"""
```

![](img/tweet_1.png)


**#1074430774017966082**
```text
@65Filippov tweeted in #1074430774017966082:
"""
RT @ntvru: Совет да любовь: в бразильском Сан-Паулу прошла массовая свадьба однополых пар https://t.co/SpeBjcEdac
"""
```

![](img/tweet_5.png)

Отобразим файлы с промежуточными ежеминутными результатами.

```bash
[cloudera@quickstart ~]$ ls -l /tmp/retweets/
total 60
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:24 counts-1544999040000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:25 counts-1544999100000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:26 counts-1544999160000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:27 counts-1544999220000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:28 counts-1544999280000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:29 counts-1544999340000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:30 counts-1544999400000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:31 counts-1544999460000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:32 counts-1544999520000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:33 counts-1544999580000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:34 counts-1544999640000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:35 counts-1544999700000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:36 counts-1544999760000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:37 counts-1544999820000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 16 14:38 counts-1544999880000
```

Выведем результаты файла, соответствующего 7 итерации.

```text
-------------------------------------------
Time: 2018-12-16 14:30:00
-------------------------------------------
(1074429629405061132, 636)
(1074430774017966082, 65)
(1074430307321946112, 28)
(1074429830840676352, 22)
(1074431121570578435, 17)
(1074431321680752641, 8)
(1074430858805895170, 6)
(1074430039129817088, 5)
(1074430010574942219, 5)
(1074430927189786624, 4)
...
```
---
```bash
[cloudera@quickstart ~]$ ls -l /tmp/retweets/counts-1544999400000/
total 4
-rw-r--r-- 1 cloudera cloudera 706 Dec 16 14:30 part-00000
-rw-r--r-- 1 cloudera cloudera   0 Dec 16 14:30 _SUCCESS
[cloudera@quickstart ~]$ cat /tmp/retweets/counts-1544999400000/part-00000 
(1074429629405061132, 636)
(1074430774017966082, 65)
(1074430307321946112, 28)
(1074429830840676352, 22)
(1074431121570578435, 17)
(1074431321680752641, 8)
(1074430858805895170, 6)
(1074430039129817088, 5)
(1074430010574942219, 5)
(1074430927189786624, 4)
(1074431240709632000, 4)
(1074430193320775681, 4)
(1074430822055317505, 4)
(1074431378312359937, 4)
(1074429845843660800, 2)
(1074430917861613568, 2)
(1074430260526104576, 1)
(1074429800834654208, 0)
(1074429950382493696, 0)
(1074430353677447168, 0)
(1074430522644971520, 0)
(1074431003207368706, 0)
(1074431204437438464, 0)
(1074430004992327683, 0)
(1074430205643636741, 0)
(1074430463446519809, 0)
(1074430537132060673, 0)
(1074431072203608065, 0)
```

Таким образом, программа верно выводит количество ретвитов пользователя **за последние 15 минут**,

при этом обновление статистики выполняется **каждую минуту** (при этом она выгружается в файл),

а общий результат **накапливается между итерациями**.

В конце верно выводится 5 наиболее популярных твитов с информацией об их авторе.
